In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
airbnb_df = pd.read_csv('Chicago Airbnb data.csv')
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,60,2,178,12/15/2019,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,7/14/2020,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,3/8/2020,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,10/23/2019,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,NaN,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2/14/2020,0.61,5,168


In [3]:
airbnb_df = airbnb_df.drop('neighbourhood_group', axis=1)
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,60,2,178,12/15/2019,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,7/14/2020,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,3/8/2020,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,10/23/2019,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2/14/2020,0.61,5,168


In [4]:
working_df = airbnb_df.drop(['id','name','host_id','host_name','reviews_per_month','neighbourhood','last_review'], axis=1)
working_df.head()

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
0,41.78790,-87.58780,Private room,60,2,178,1,353
1,41.85495,-87.69696,Entire home/apt,105,2,395,1,155
2,41.90289,-87.68182,Entire home/apt,60,2,384,1,321
3,41.91769,-87.63788,Entire home/apt,65,4,49,9,300
4,41.79612,-87.59261,Private room,21,1,44,5,168


In [5]:
X = pd.get_dummies(working_df)
X

,latitude,longitude,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,41.78790,-87.58780,60,2,178,1,353,0,0,1,0
1,41.85495,-87.69696,105,2,395,1,155,1,0,0,0
2,41.90289,-87.68182,60,2,384,1,321,1,0,0,0
3,41.91769,-87.63788,65,4,49,9,300,1,0,0,0
4,41.79612,-87.59261,21,1,44,5,168,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
6392,41.85574,-87.62507,146,2,0,4,180,1,0,0,0
6393,41.85561,-87.62342,195,2,0,4,180,1,0,0,0
6394,41.85584,-87.62404,493,2,0,4,177,1,0,0,0
6395,41.89005,-87.66668,53,32,0,1,83,0,0,1,0


In [6]:
X = X.drop('price', axis=1)
y = airbnb_df.price

In [12]:
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[-1.88632217,  1.79769864, -0.26829748, ..., -0.10743988,
         1.577944  , -0.11945651],
       [-0.74841388, -0.77616029, -0.26829748, ..., -0.10743988,
        -0.63373605, -0.11945651],
       [ 0.06517781, -0.41917766, -0.26829748, ..., -0.10743988,
        -0.63373605, -0.11945651],
       ...,
       [-0.73330966,  0.94320388, -0.26829748, ..., -0.10743988,
        -0.63373605, -0.11945651],
       [-0.15273035, -0.06219503,  1.04835391, ..., -0.10743988,
         1.577944  , -0.11945651],
       [-1.17795093,  1.02714432, -0.31218586, ..., -0.10743988,
        -0.63373605, -0.11945651]])

In [13]:
model = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

In [14]:
model.fit(X_train, y_train)

LinearRegression()

In [15]:
model.score(X_test, y_test)

0.005901109980655739